In [2]:

import tensorflow as tf

with tf.device('/gpu:0'):
    a = tf.zeros([1000, 1000])
    print(a.device)

/device:GPU:0


In [1]:
import tensorflow as tf


def get_all_values_except_specified(tensor, excluded):
    with tf.name_scope('get_all_values_except_specified'):
        tensor = tf.reshape(tensor, [-1])
        excluded = tf.reshape(excluded, [-1])
        excluded_shape = tf.shape(excluded)
        tensor_expanded = tf.reshape(tensor, [-1, 1])
        multiples = tf.concat([[1], excluded_shape], 0)
        tensor_expanded = tf.tile(tensor_expanded, multiples)
        masks = tf.cast(tf.equal(tf.cast(tensor_expanded, tf.int32), tf.cast(excluded, tf.int32)), tf.int32)
        mask = tf.reduce_sum(masks, [1])
        mask = tf.cast(tf.cast(mask, dtype=tf.bool), dtype=tf.int32) - 1
        return tf.boolean_mask(tensor, mask)
    

tensor, num_dims, axes, output = (
                    [[[1, 2], [3, 4]], [[5, 6], [7, 8]]],

                    5,

                    [0, 2, 4],

                    [[[[[1, 2]], [[3, 4]]], [[[5, 6]], [[7, 8]]]]],
                )

if not tf.contrib.framework.is_tensor(tensor):
    tensor = tf.constant(tensor)
if not tf.contrib.framework.is_tensor(axes):
    axes = tf.constant(axes, dtype=tf.int32)
sh = tf.shape(tensor, out_type=tf.int32)
nd = tf.shape(sh, out_type=tf.int32)[0]
assert_axes_smaller_than_num_dims = tf.assert_less(
    axes, num_dims, message='`axes` has to be less than `num_dims`')
check_num_dims = tf.assert_greater_equal(
    num_dims, nd,
    message='`num_dims` has to be greater or equal to number of dimensions in `tensor`'
)
ass_axes_bigger_or_equal_than_num_dims = tf.assert_greater_equal(axes, -num_dims)

negative_axes_mask = tf.cast(axes < 0, tf.int32)
axes += negative_axes_mask * num_dims

ones_for_expansion = tf.ones(tf.reshape(num_dims - nd, [1]), dtype=tf.int32)
shape_for_expansion = tf.concat([sh, ones_for_expansion], 0)

tensor = tf.reshape(tensor, shape_for_expansion)

# remained_axes = get_all_values_except_specified(tf.range(num_dims, dtype=tf.int32), axes)
# perm = tf.concat([axes, remained_axes], 0)
updates = tf.range(0, num_dims, 1, dtype=tf.int32)
remained_positions = get_all_values_except_specified(tf.range(num_dims, dtype=tf.int32), axes)
indices = tf.concat([axes, remained_positions], 0)
indices = tf.reshape(indices, [-1, 1])
perm_shape = tf.reshape(num_dims, [1])
perm = tf.scatter_nd(indices, updates, perm_shape)

with tf.control_dependencies([check_num_dims, assert_axes_smaller_than_num_dims, ass_axes_bigger_or_equal_than_num_dims]):
    tensor = tf.transpose(tensor, perm=perm)
    

with tf.Session() as sess:
    print(sess.run(tensor))


For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.

[[[[[1 2]]

   [[3 4]]]]



 [[[[5 6]]

   [[7 8]]]]]


In [1]:
import numpy as np

from helmo.util.scripts.text_neuron_correlation import compute_stats

num_unrollings = 20
num_units = 4

# a = np.random.rand(num_units, num_unrollings)

m = np.random.choice([-1, 1, 0], num_unrollings)
a = np.stack([m]*num_units, 1)
print(a)

stats = compute_stats(a, m)
print(stats)

[[-1 -1 -1 -1]
 [ 1  1  1  1]
 [ 0  0  0  0]
 [-1 -1 -1 -1]
 [ 1  1  1  1]
 [-1 -1 -1 -1]
 [ 1  1  1  1]
 [-1 -1 -1 -1]
 [ 0  0  0  0]
 [ 0  0  0  0]
 [-1 -1 -1 -1]
 [-1 -1 -1 -1]
 [-1 -1 -1 -1]
 [-1 -1 -1 -1]
 [ 0  0  0  0]
 [ 1  1  1  1]
 [ 0  0  0  0]
 [ 1  1  1  1]
 [ 0  0  0  0]
 [ 1  1  1  1]]
{'markup': array([-1,  1,  0, -1,  1, -1,  1, -1,  0,  0, -1, -1, -1, -1,  0,  1,  0,
        1,  0,  1]), 'relevant_markup': [-1, 1, -1, 1, -1, 1, -1, -1, -1, -1, -1, 1, 1, 1], 'relevant_activations': array([[-1, -1, -1, -1],
       [ 1,  1,  1,  1],
       [-1, -1, -1, -1],
       [ 1,  1,  1,  1],
       [-1, -1, -1, -1],
       [ 1,  1,  1,  1],
       [-1, -1, -1, -1],
       [-1, -1, -1, -1],
       [-1, -1, -1, -1],
       [-1, -1, -1, -1],
       [-1, -1, -1, -1],
       [ 1,  1,  1,  1],
       [ 1,  1,  1,  1],
       [ 1,  1,  1,  1]]), 'matches': array([[0.75      , 0.75      , 0.75      , 0.75      ],
       [1.33333333, 1.33333333, 1.33333333, 1.33333333],
       [0.75      , 

In [1]:
0 == None

False

In [3]:
import queue
import Multiprocessing

def target_function(q, *args, **kwargs):
    print("in target function")
    res = func(*args, **kwargs)
    q.put(res)
    print("res is put in queue")

def wrapper(*args, **kwargs):
    print("in wrapper")
    q = queue.Queue()
    p = multiprocessing.Process(target=target_function, args=tuple([q]+list(args)), kwargs=kwargs)
    p.start()
    res = q.get()
    print("got result")
    p.join()
    print("joined")
    return res

def func():
    return 1

wrapper()

in wrapper


NameError: name 'queue' is not defined

In [13]:
import tensorflow as tf

print(tf.placeholder(tf.float32).__class__.__bases__)
print(tf.zeros([1]).__class__.__bases__)
print(tf.Variable(0).__class__.__bases__[0].__bases__[0].__bases__)

(<class 'tensorflow.python.framework.ops._TensorLike'>,)
(<class 'tensorflow.python.framework.ops._TensorLike'>,)
(<class 'tensorflow.python.training.checkpointable.base.CheckpointableBase'>,)


In [1]:
def partition(A, lo, hi):
    #print(lo, hi, A)
    if A[lo] > A[hi]:
        A[lo], A[hi] = A[hi], A[lo]
    pivot_1 = A[lo]
    pivot_2 = A[hi]
    i_1 = lo + 1
    i_2 = hi - 1
    for j in range(lo + 1, hi):
        # print(j, i_1, i_2, A)
        if A[j] < pivot_1:
            A[i_1], A[j] = A[j], A[i_1]
            i_1 += 1
        elif A[j] > pivot_2:
            while (A[i_2] > pivot_2) and (i_2 > j):
                i_2 -= 1
            A[i_2], A[j] = A[j], A[i_2]
            i_2 -= 1
            
            if A[j] < pivot_1:
                A[i_1], A[j] = A[j], A[i_1]
                i_1 += 1

        if j >= i_2:
            break
    i_1 -= 1
    i_2 += 1
    A[i_1], A[lo] = A[lo], A[i_1]
    A[i_2], A[hi] = A[hi], A[i_2]
    #print(i_1, i_2, A)
    return i_1, i_2

def DualPivotQuickSort(A, lo, hi):
    if lo < hi:
        p_1, p_2 = partition(A, lo, hi)
        DualPivotQuickSort(A, lo, p_1 - 1)
        DualPivotQuickSort(A, p_1 + 1, p_2 - 1)
        DualPivotQuickSort(A, p_2 + 1, hi)


In [5]:
import random



for _ in range(100000):
    A = [random.randint(0, 20) for _ in range(20)]
    a = A.copy()
    DualPivotQuickSort(a, 0, len(A) - 1)
    assert sorted(A) == a


In [6]:

def union(A, B):
    C = []
    i_1, i_2 = 0, 0
    for _ in range(len(A) + len(B)):
        
        if i_1 >= len(A):
            C += B[i_2:]
            break
        
        if i_2 >= len(B):
            C += A[i_1:]
            break
            
        if A[i_1] < B[i_2]:
            C.append(A[i_1])
            i_1 += 1
        elif A[i_1] > B[i_2]:
            C.append(B[i_2])
            i_2 += 1
        else:
            C.append(A[i_1])
            i_1 += 1
            i_2 += 1
            
    return C

In [8]:
union([ 3, 5, 6, 7], [1, 3])

[1, 3, 5, 6, 7]

In [12]:
def partition(A,lo,hi): 
    
   
    if lo < hi:
        if A[lo] > A[hi]:
            A[lo],A[hi] = A[hi], A[lo]
        lp = lo + 1
        rp = hi - 1 
        piv1 = A[lo]
        piv2 = A[hi]  
            
        for j in range(lo + 1, hi):
        # print(j, i_1, i_2, A)
            if A[j] < piv1:
                A[lp], A[j] = A[j], A[lp]
                lp += 1
            elif A[j] > piv2:
                while (A[rp] > piv2) and (rp > j):
                    rp -= 1
                A[rp], A[j] = A[j], A[rp]
                rp -= 1

            if A[j] < piv1:
                    A[lp], A[j] = A[j], A[lp]
                    lp += 1

            if j >= rp:
                break
        lp -= 1
        rp += 1
        A[lp], A[lo] = A[lo], A[lp]
        A[rp], A[hi] = A[hi], A[rp]
        #print(i_1, i_2, A)
        return lp, rp

            
def DualPivotQuickSort(A, lo, hi):
   # def partition(arr, low, high, lp):
        
    if lo < hi: 
        #// lp means left pivot, and rp means right pivot. 
        #lp, rp  
        p1,p2 = partition(A, lo, hi)
        DualPivotQuickSort(A, lo, p1 - 1)
        DualPivotQuickSort(A, p1 + 1, p2 - 1)
        DualPivotQuickSort(A, p2 + 1, hi)
   # return a

num_tests = 1000
for _ in range(num_tests):
    try:
        a = [random.randint(0, 100) for _ in range(10)]
        a1 = a.copy()
        DualPivotQuickSort(a, 0, len(a)-1)
        if a != sorted(a1):
            print(a1)
    except IndexError:
        print(a1)
        break

[83, 51, 94, 75, 23, 32, 25, 44, 89, 90]
